In [1]:
import numpy as np
import pandas as pd
import scipy
from tqdm import tqdm
import sklearn
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer
from sklearn.linear_model import LassoCV, BayesianRidge, LogisticRegression, LogisticRegressionCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, RobustScaler, MinMaxScaler
from sklearn.model_selection import RepeatedStratifiedKFold, StratifiedKFold, GridSearchCV
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.metrics import f1_score, roc_auc_score, brier_score_loss
from sklearn.svm import SVC
from xgboost import XGBClassifier

sklearn.set_config(enable_metadata_routing=False)

In [3]:
#import heavy file
connectome = pd.read_csv('./TRAIN_NEW/TRAIN_FUNCTIONAL_CONNECTOME_MATRICES_new_36P_Pearson.csv')

In [4]:
SEED = 42
REPEATS = 5
FOLDS = 5

# Import Data 
# Train
quantitative_metadata_train = pd.read_excel("./TRAIN_NEW/TRAIN_QUANTITATIVE_METADATA_new.xlsx")
categorical_metadata_train = pd.read_excel("./TRAIN_NEW/TRAIN_CATEGORICAL_METADATA_new.xlsx")
training_solutions = pd.read_excel("./TRAIN_NEW/TRAINING_SOLUTIONS.xlsx")

quantitative_metadata_train = quantitative_metadata_train.sort_values(by='participant_id')
categorical_metadata_train = categorical_metadata_train.sort_values(by='participant_id')
training_solutions  = training_solutions.sort_values(by='participant_id')

# Test
quantitative_metadata_test = pd.read_excel("./TEST/TEST_QUANTITATIVE_METADATA.xlsx")
categorical_metadata_test = pd.read_excel("./TEST/TEST_CATEGORICAL.xlsx")

quantitative_metadata_test = quantitative_metadata_test.sort_values(by='participant_id')
categorical_metadata_test = categorical_metadata_test.sort_values(by='participant_id')

#Dictionary
data_dictionary = pd.read_excel('./Data Dictionary.xlsx', sheet_name=None)['Dictionary']

#Select Columns which have consistent train test data
drop_cat_cols = [] 
for cat in categorical_metadata_train.columns:
    if cat not in ['participant_id']:
        if False == np.all(np.isin(categorical_metadata_test[cat].dropna().unique(), categorical_metadata_train[cat].dropna().unique())):
            drop_cat_cols.append(cat)
        print(cat,np.all(np.isin(
                            categorical_metadata_test[cat].dropna().unique(), 
                            categorical_metadata_train[cat].dropna().unique()
                         )))

#drop the columns
categorical_metadata_train = categorical_metadata_train.drop(drop_cat_cols, axis=1)
categorical_metadata_test = categorical_metadata_test.drop(drop_cat_cols, axis=1)

Basic_Demos_Enroll_Year False
Basic_Demos_Study_Site False
PreInt_Demos_Fam_Child_Ethnicity True
PreInt_Demos_Fam_Child_Race True
MRI_Track_Scan_Location True
Barratt_Barratt_P1_Edu True
Barratt_Barratt_P1_Occ True
Barratt_Barratt_P2_Edu True
Barratt_Barratt_P2_Occ True


In [5]:


# Separate features and target
quantitative_features_train = quantitative_metadata_train.drop(columns=['participant_id'])
categorical_features_train = categorical_metadata_train.drop(columns=['participant_id'])
quantitative_features_test = quantitative_metadata_test.drop(columns=['participant_id'])
categorical_features_test = categorical_metadata_test.drop(columns=['participant_id'])

# Column names
quantitative_cols = quantitative_features_train.columns.tolist()
categorical_cols = categorical_features_train.columns.tolist()

# Build column transformer with imputation, scaling, and encoding
numeric_pipeline = Pipeline([
    ('imputer', IterativeImputer(estimator=BayesianRidge(), max_iter=50, random_state=SEED)),
    ('scaler', StandardScaler())
])

categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(drop='first', sparse_output=False))
])

# Full preprocessor
preprocessor = ColumnTransformer([
    ('num', numeric_pipeline, quantitative_cols),
    ('cat', categorical_pipeline, categorical_cols)
])

# Combine all features
combined_features_train = pd.concat([quantitative_features_train, categorical_features_train], axis=1)
combined_features_test = pd.concat([quantitative_features_test, categorical_features_test], axis=1)

train_fe = combined_features_train.copy()
test_fe = combined_features_test.copy()

for df in [train_fe, test_fe]:
    df['log_age'] = np.log1p(df['MRI_Track_Age_at_Scan'])

bins = [0, 8, 10, 12, 14, 18]
labels_age = ['<8', '8-10', '10-12', '12-14', '14-18']
for df in [train_fe, test_fe]:
    df['age_group'] = pd.cut(df['MRI_Track_Age_at_Scan'], bins=bins, labels=labels_age)
    df['age_group'] = df['age_group'].cat.codes  # convert to numeric

for df in [train_fe, test_fe]:
    df['hyper_prosocial_ratio'] = df['SDQ_SDQ_Hyperactivity'] / (df['SDQ_SDQ_Prosocial'] + 0.01)
    df['conduct_emotion_ratio'] = df['SDQ_SDQ_Conduct_Problems'] / (df['SDQ_SDQ_Emotional_Problems'] + 0.01)
    df['external_internal_ratio'] = df['SDQ_SDQ_Externalizing'] / (df['SDQ_SDQ_Internalizing'] + 0.01)

for df in [train_fe, test_fe]:
    df['parent_edu_avg'] = (df['Barratt_Barratt_P1_Edu'] + df['Barratt_Barratt_P2_Edu']) / 2
    df['parent_edu_diff'] = np.abs(df['Barratt_Barratt_P1_Edu'] - df['Barratt_Barratt_P2_Edu'])

combined_features_train = train_fe.copy()
combined_features_test = test_fe.copy()

categorical_cols.extend(['age_group'])
quantitative_cols.extend(['parent_edu_diff','parent_edu_avg','external_internal_ratio','conduct_emotion_ratio','hyper_prosocial_ratio','log_age'])

In [6]:
# Apply transformations
processed_array_train = preprocessor.fit_transform(combined_features_train)
processed_array_test = preprocessor.transform(combined_features_test)

# Get encoded feature names
encoded_cat_cols = preprocessor.named_transformers_['cat']['encoder'].get_feature_names_out(categorical_cols)
final_feature_names = quantitative_cols + list(encoded_cat_cols)

# Create DataFrame
processed_df_train = pd.DataFrame(processed_array_train, columns=final_feature_names)
processed_df_test = pd.DataFrame(processed_array_test, columns=final_feature_names)

processed_df_train['participant_id'] = quantitative_metadata_train['participant_id']
processed_df_test['participant_id'] = quantitative_metadata_test['participant_id']

train_combined = pd.merge(processed_df_train, training_solutions, on='participant_id', how='inner')
test_combined = processed_df_test
display(train_combined.head())
#---------------------------------------------------------------------
y_adhd = train_combined["ADHD_Outcome"]
y_sex = train_combined["Sex_F"]
#---------------------------------------------------------------------
combinations = train_combined["ADHD_Outcome"].astype(str) + train_combined["Sex_F"].astype(str)

,EHQ_EHQ_Total,ColorVision_CV_Score,APQ_P_APQ_P_CP,APQ_P_APQ_P_ID,APQ_P_APQ_P_INV,APQ_P_APQ_P_OPD,APQ_P_APQ_P_PM,APQ_P_APQ_P_PP,SDQ_SDQ_Conduct_Problems,SDQ_SDQ_Difficulties_Total,...,Barratt_Barratt_P2_Occ_40.0,Barratt_Barratt_P2_Occ_45.0,age_group_0.0,age_group_1.0,age_group_2.0,age_group_3.0,age_group_4.0,participant_id,ADHD_Outcome,Sex_F
0,0.818649,-0.200432,-0.619974,0.466855,0.871365,-1.195780,0.676349,0.492205,0.459264,0.737480,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,00aIpNTbG5uh,1,0
1,0.662351,0.277619,-0.619974,-0.372214,-0.984879,2.118612,2.233865,1.479493,1.451999,1.199700,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,00fV0OyyoLfw,1,0
2,0.549121,0.277619,-0.619974,2.144992,-0.572380,0.009453,1.844486,0.821301,0.459264,1.815994,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,04X1eiS79T4B,0,1
3,0.683986,0.277619,-0.619974,-0.651903,0.458866,-0.894472,0.676349,0.821301,-1.029839,-1.111402,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,05ocQutkURd6,0,1
4,-1.203320,0.277619,3.157463,-0.372214,-0.984879,1.214687,-0.881168,-0.495084,1.948367,1.661920,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,06YUNBA9ZRLq,1,0


In [7]:
features_sex_old = [
       'EHQ_EHQ_Total', 'ColorVision_CV_Score', 'APQ_P_APQ_P_CP',
       'APQ_P_APQ_P_ID', 'APQ_P_APQ_P_INV', 'APQ_P_APQ_P_OPD',
       'APQ_P_APQ_P_PM', 'APQ_P_APQ_P_PP', 'SDQ_SDQ_Conduct_Problems',
       'SDQ_SDQ_Difficulties_Total', 'SDQ_SDQ_Emotional_Problems',
       'SDQ_SDQ_Externalizing', 'SDQ_SDQ_Generating_Impact',
       'SDQ_SDQ_Hyperactivity', 'SDQ_SDQ_Internalizing',
       'SDQ_SDQ_Peer_Problems', 'SDQ_SDQ_Prosocial', 'MRI_Track_Age_at_Scan',
       'Barratt_Barratt_P1_Edu', 'Barratt_Barratt_P2_Edu', 'log_age', 'age_group',
       'hyper_prosocial_ratio', 'conduct_emotion_ratio',
       'external_internal_ratio',
       'parent_edu_avg', 'parent_edu_diff'
]

features_sex = [s for s in train_combined.columns if any(sub in s for sub in features_sex_old)]


features_adhd_old = [
       'EHQ_EHQ_Total', 'ColorVision_CV_Score', 'APQ_P_APQ_P_CP',
       'APQ_P_APQ_P_ID', 'APQ_P_APQ_P_INV', 'APQ_P_APQ_P_OPD',
       'APQ_P_APQ_P_PM', 'APQ_P_APQ_P_PP', 'SDQ_SDQ_Conduct_Problems',
       'SDQ_SDQ_Difficulties_Total', 'SDQ_SDQ_Emotional_Problems',
       'SDQ_SDQ_Externalizing', 'SDQ_SDQ_Generating_Impact',
       'SDQ_SDQ_Hyperactivity', 'SDQ_SDQ_Internalizing',
       'SDQ_SDQ_Peer_Problems', 'SDQ_SDQ_Prosocial', 'MRI_Track_Age_at_Scan',
       'Barratt_Barratt_P1_Edu', 'Barratt_Barratt_P2_Edu', 'sex_proba',
       'I_APQ_P_APQ_P_INV', 'I_APQ_P_APQ_P_PP', 'I_SDQ_SDQ_Hyperactivity',
       'I_MRI_Track_Age_at_Scan', 'I_SDQ_SDQ_Generating_Impact', 'log_age', 'age_group',
       'hyper_prosocial_ratio', 'conduct_emotion_ratio',
       'external_internal_ratio',
       'parent_edu_avg', 'parent_edu_diff'
]

features_adhd = [s for s in train_combined.columns if any(sub in s for sub in features_adhd_old)]


interactions = [
    "APQ_P_APQ_P_INV", "APQ_P_APQ_P_PP", "SDQ_SDQ_Hyperactivity",
    "MRI_Track_Age_at_Scan", "SDQ_SDQ_Generating_Impact"
]

In [8]:
def eval_metrics(y_true, y_pred, weights, label="None", thresh=0.5):
    """Evaluate predictions using Brier Score and F1 Score."""
    brier = brier_score_loss(y_true, y_pred)
    f1 = f1_score(y_true, (y_pred > thresh).astype(int), sample_weight=weights)
    print(f"{label} -> Brier Score: {brier:.4f}, F1: {f1:.4f}")
    return brier, f1

#placeholder for results
scores_sex = []
scores_adhd = []

sex_oof = np.zeros(len(y_sex))
adhd_oof = np.zeros(len(y_adhd))

t_sex = 0.3
t_adhd = 0.4

rskf = RepeatedStratifiedKFold(n_splits=FOLDS, n_repeats=REPEATS, random_state=SEED)
skf = StratifiedKFold(n_splits=FOLDS)

In [9]:

# Define candidate models and their parameters (including XGBoost)
params_xgb = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    # 'subsample': 0.8,
    # 'colsample_bytree': 0.8,
    'random_state': SEED
}

model_params = {
    'logistic_regression': {
        'model': LogisticRegression(solver='saga', max_iter=5000),
        'params': {
            'penalty': ['l1', 'l2'],
            'C': [0.01, 0.1, 1, 10]
        },
        'weight_param': 'clf__sample_weight'
    },
    'svc': {
        'model': SVC(probability=True),
        'params': {
            'C': [0.1, 1, 10],
            'kernel': ['linear', 'rbf']
        },
        'weight_param': 'clf__sample_weight'
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params': {
            'n_estimators': [50, 100],
            'max_depth': [5, 10, None]
        },
        'weight_param': 'clf__sample_weight'
    },
    'xgboost': {
        'model': XGBClassifier(**params_xgb),
        'params': {

            'n_estimators': [300, 150, 50, 100],
            'max_depth': [3,5],
            'learning_rate': [0.03, 0.01, 0.07],
            'subsample': [0.7,0.8, 1.0],
            'colsample_bytree': [0.7,0.8, 1.0]
        },
        'weight_param': 'clf__sample_weight'
    }
}


def GridSearch(X_grid, y_grid, sample_weights):
    scores = []
    # GridSearchCV for each model
    for model_name, mp in model_params.items():
        pipe = Pipeline([
            ('scaler', StandardScaler()),
            ('clf', mp['model'])
        ])

        clf = GridSearchCV(
            pipe,
            {f'clf__{k}': v for k, v in mp['params'].items()},
            cv=skf,
            scoring='f1',
            return_train_score=False
        )
        fit_params = {mp['weight_param']: sample_weights} if 'weight_param' in mp else {}
        clf.fit(X_grid, y_grid, **fit_params)

        scores.append({
            'model': model_name,
            'best_score': clf.best_score_,
            'best_params': clf.best_params_,
            'best_estimator': clf.best_estimator_
        })

    # Create DataFrame and select best
    results_df = pd.DataFrame(scores)
    results_df_sorted = results_df.sort_values(by='best_score', ascending=False).reset_index(drop=True)
    best_model = results_df_sorted.loc[0, 'best_estimator']
    print(f"Best Model: {results_df_sorted.loc[0, 'model']}")
    print(f"Best F1 Score: {results_df_sorted.loc[0, 'best_score']:.4f}")
    print(f"Best Params: {results_df_sorted.loc[0, 'best_params']}")

    return best_model , results_df_sorted.loc[0, 'best_score']

In [10]:
all_Bestmodels_sex = []
all_Bestmodels_adhd = []
scores_sex_gridcvs = []
scores_adhd_gridcvs =[]
scores_adhd = []
scores_sex = []

for fold, (train_idx, val_idx) in enumerate(rskf.split(train_combined, combinations), 1):
    print(f"\n=== Fold {fold} ===")

    X_train, X_val = train_combined.iloc[train_idx].copy(), train_combined.iloc[val_idx].copy()
    y_train_adhd, y_val_adhd = y_adhd.iloc[train_idx], y_adhd.iloc[val_idx]
    y_train_sex, y_val_sex = y_sex.iloc[train_idx], y_sex.iloc[val_idx]
    weights_train = np.where(combinations.iloc[train_idx]=="11", 2, 1)
    weights = np.where(combinations.iloc[val_idx]=="11", 2, 1)

    model_1, scores_sex_gridcv = GridSearch(X_train[features_sex], y_train_sex, weights_train)
    # model_1.fit(X_train[features_sex], y_train_sex, sample_weight=weights_train)

    all_Bestmodels_sex.append(model_1)
    scores_sex_gridcvs.append(scores_sex_gridcv)

    sex_train = model_1.predict_proba(X_train[features_sex])[:, 1]
    sex_val = model_1.predict_proba(X_val[features_sex])[:, 1]
    sex_oof[val_idx] += sex_val / REPEATS

    sex_brier, sex_f1 = eval_metrics(y_val_sex, sex_val, weights, "Sex_F", thresh=t_sex)
    scores_sex.append((sex_brier, sex_f1))

    X_train.loc[:, "sex_proba"] = sex_train
    X_val.loc[:, "sex_proba"] = sex_val

    for interaction in interactions:
        X_train.loc[:, f"I_{interaction}"] = X_train[interaction] * X_train["sex_proba"]
        X_val.loc[:, f"I_{interaction}"] = X_val[interaction] * X_val["sex_proba"]

    model_2, scores_adhd_gridcv = GridSearch(X_train[features_adhd], y_train_adhd, weights_train)
    all_Bestmodels_adhd.append(model_2)
    scores_adhd_gridcvs.append(scores_adhd_gridcv)

    adhd_val = model_2.predict_proba(X_val[features_adhd])[:, 1]
    adhd_oof[val_idx] += adhd_val / REPEATS

    adhd_brier, adhd_f1 = eval_metrics(y_val_adhd, adhd_val, weights, "Outcome ADHD", thresh=t_adhd)
    scores_adhd.append((adhd_brier, adhd_f1))

print(f"\n=== CV Results ===")
print(f"Sex Mean Brier Score: {np.mean([s[0] for s in scores_sex]):.4f}")
print(f"Sex Mean F1: {np.mean([s[1] for s in scores_sex]):.4f}")
print(f"ADHD Mean Brier Score: {np.mean([s[0] for s in scores_adhd]):.4f}")
print(f"ADHD Mean F1: {np.mean([s[1] for s in scores_adhd]):.4f}")


=== Fold 1 ===
Best Model: logistic_regression
Best F1 Score: 0.4728
Best Params: {'clf__C': 10, 'clf__penalty': 'l2'}
Sex_F -> Brier Score: 0.2401, F1: 0.5706
Best Model: xgboost
Best F1 Score: 0.8637
Best Params: {'clf__colsample_bytree': 0.7, 'clf__learning_rate': 0.03, 'clf__max_depth': 3, 'clf__n_estimators': 100, 'clf__subsample': 1.0}
Outcome ADHD -> Brier Score: 0.1253, F1: 0.8977

=== Fold 2 ===


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Best Model: svc
Best F1 Score: 0.4867
Best Params: {'clf__C': 10, 'clf__kernel': 'linear'}
Sex_F -> Brier Score: 0.2211, F1: 0.5696
Best Model: xgboost
Best F1 Score: 0.8676
Best Params: {'clf__colsample_bytree': 0.7, 'clf__learning_rate': 0.07, 'clf__max_depth': 3, 'clf__n_estimators': 50, 'clf__subsample': 0.8}
Outcome ADHD -> Brier Score: 0.1523, F1: 0.8802

=== Fold 3 ===
Best Model: logistic_regression
Best F1 Score: 0.4615
Best Params: {'clf__C': 0.1, 'clf__penalty': 'l1'}
Sex_F -> Brier Score: 0.2298, F1: 0.6350
Best Model: xgboost
Best F1 Score: 0.8652
Best Params: {'clf__colsample_bytree': 0.7, 'clf__learning_rate': 0.03, 'clf__max_depth': 3, 'clf__n_estimators': 100, 'clf__subsample': 0.8}
Outcome ADHD -> Brier Score: 0.1519, F1: 0.8753

=== Fold 4 ===


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Best Model: xgboost
Best F1 Score: 0.4757
Best Params: {'clf__colsample_bytree': 1.0, 'clf__learning_rate': 0.07, 'clf__max_depth': 3, 'clf__n_estimators': 50, 'clf__subsample': 0.7}
Sex_F -> Brier Score: 0.2292, F1: 0.6069
Best Model: xgboost
Best F1 Score: 0.8730
Best Params: {'clf__colsample_bytree': 0.8, 'clf__learning_rate': 0.07, 'clf__max_depth': 3, 'clf__n_estimators': 150, 'clf__subsample': 1.0}
Outcome ADHD -> Brier Score: 0.1604, F1: 0.8722

=== Fold 5 ===


...

Best Model: logistic_regression
Best F1 Score: 0.4866
Best Params: {'clf__C': 10, 'clf__penalty': 'l2'}
Sex_F -> Brier Score: 0.2333, F1: 0.6075
Best Model: xgboost
Best F1 Score: 0.8662
Best Params: {'clf__colsample_bytree': 1.0, 'clf__learning_rate': 0.07, 'clf__max_depth': 3, 'clf__n_estimators': 50, 'clf__subsample': 0.7}
Outcome ADHD -> Brier Score: 0.1527, F1: 0.8774

=== Fold 23 ===


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Best Model: svc
Best F1 Score: 0.4868
Best Params: {'clf__C': 10, 'clf__kernel': 'linear'}
Sex_F -> Brier Score: 0.2185, F1: 0.6261
Best Model: xgboost
Best F1 Score: 0.8695
Best Params: {'clf__colsample_bytree': 0.8, 'clf__learning_rate': 0.07, 'clf__max_depth': 3, 'clf__n_estimators': 50, 'clf__subsample': 1.0}
Outcome ADHD -> Brier Score: 0.1485, F1: 0.8722

=== Fold 24 ===
Best Model: svc
Best F1 Score: 0.4556
Best Params: {'clf__C': 10, 'clf__kernel': 'linear'}
Sex_F -> Brier Score: 0.2107, F1: 0.6185
Best Model: xgboost
Best F1 Score: 0.8636
Best Params: {'clf__colsample_bytree': 0.7, 'clf__learning_rate': 0.03, 'clf__max_depth': 5, 'clf__n_estimators': 100, 'clf__subsample': 0.7}
Outcome ADHD -> Brier Score: 0.1395, F1: 0.8950

=== Fold 25 ===


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Best Model: logistic_regression
Best F1 Score: 0.4655
Best Params: {'clf__C': 0.1, 'clf__penalty': 'l2'}
Sex_F -> Brier Score: 0.2328, F1: 0.6234
Best Model: logistic_regression
Best F1 Score: 0.8660
Best Params: {'clf__C': 0.1, 'clf__penalty': 'l1'}
Outcome ADHD -> Brier Score: 0.1540, F1: 0.8720

=== CV Results ===
Sex Mean Brier Score: 0.2270
Sex Mean F1: 0.6147
ADHD Mean Brier Score: 0.1476
ADHD Mean F1: 0.8818


In [11]:
# === 1. Find Best Threshold for Sex Based on OOF Probabilities ===
weights = ((y_adhd == 1) & (y_sex == 1)) + 1  # Double weight if both ADHD and Sex == 1

thresholds = np.linspace(0, 1, 100)
sex_scores = []

for t in tqdm(thresholds, desc="Sex Thresholds"):
    tmp_pred = np.where(sex_oof > t, 1, 0)
    tmp_score = f1_score(y_sex, tmp_pred, sample_weight=weights)
    sex_scores.append(tmp_score)

best_sex_threshold = thresholds[np.argmax(sex_scores)]
print(len(all_Bestmodels_sex),len(scores_sex))
best_sex_score = max(sex_scores)

print("Best sex threshold:", best_sex_threshold)
print("Number of evaluated thresholds:", len(sex_scores))
print("Final best sex score:", best_sex_score)


# === 2. Get Best Sex Model from CV ===
f1_scores_sex = [score[1] for score in scores_sex]
best_model_idx_sex = np.argmax(f1_scores_sex)
best_model_sex = all_Bestmodels_sex[best_model_idx_sex]

print("Best model index for Sex:", best_model_idx_sex)
print("Best F1-score for Sex:", f1_scores_sex[best_model_idx_sex])


# === 3. Get Best ADHD Model from CV ===
f1_scores_adhd = [score[1] for score in scores_adhd]
best_model_idx_adhd = np.argmax(f1_scores_adhd)
print(len(all_Bestmodels_adhd),len(scores_adhd))
best_model_adhd = all_Bestmodels_adhd[best_model_idx_adhd]

print("Best model index for ADHD:", best_model_idx_adhd)
print("Best F1-score for ADHD:", f1_scores_adhd[best_model_idx_adhd])

Sex Thresholds: 100%|███████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 181.81it/s]

25 25
Best sex threshold: 0.15151515151515152
Number of evaluated thresholds: 100
Final best sex score: 0.6299287410926366
Best model index for Sex: 6
Best F1-score for Sex: 0.6666666666666666
25 25
Best model index for ADHD: 9
Best F1-score for ADHD: 0.9071274298056156


In [12]:
# Compute weighted F1-scores to find best threshold for sex
weights = ((y_adhd == 1) & (y_sex == 1)) + 1
thresholds = np.linspace(0, 1, 100)
sex_scores = []
for t in tqdm(thresholds, desc="Sex Thresholds"):
    tmp_pred = np.where(sex_oof > t, 1, 0)
    tmp_score = f1_score(y_sex, tmp_pred, sample_weight=weights)
    sex_scores.append(tmp_score)

best_sex_threshold = thresholds[np.argmax(sex_scores)]
best_sex_score = max(sex_scores)

print("Best sex threshold:", best_sex_threshold)
print("Number of evaluated thresholds:", len(sex_scores))
print("Final best sex score:", best_sex_score)

# Ensure 'scores_sex' contains tuples (threshold, F1-score)
# print("Scores for sex grid CV:", scores_sex)
f1_scores_sex = [score[1] for score in scores_sex]

best_model_idx_sex = np.argmax(f1_scores_sex)
print("Best threshold index for Sex:", best_model_idx_sex)
print("Best F1-score for Sex:", f1_scores_sex[best_model_idx_sex])
# print("Best threshold for Sex:", scores_sex[best_model_idx_sex][0])

best_model_sex = all_Bestmodels_sex[best_model_idx_sex]
# print("Best model for Sex:", best_model_sex)
# print("Scores for ADHD grid CV:", scores_adhd)
f1_scores_adhd = [score[1] for score in scores_adhd]

best_model_idx_adhd = np.argmax(scores_adhd_gridcvs)
print("Best threshold index for ADHD:", best_model_idx_adhd)
print("Best F1-score for ADHD:", scores_adhd_gridcvs[best_model_idx_adhd])
print("Best threshold for ADHD:", scores_adhd[best_model_idx_adhd][0])

best_model_adhd = all_Bestmodels_adhd[best_model_idx_adhd]
print("Best Model for ADHD:" , best_model_adhd)

Sex Thresholds: 100%|███████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 203.99it/s]


Best sex threshold: 0.15151515151515152
Number of evaluated thresholds: 100
Final best sex score: 0.6299287410926366
Best threshold index for Sex: 6
Best F1-score for Sex: 0.6666666666666666
Best threshold index for ADHD: 8
Best F1-score for ADHD: 0.8750552685364517
Best threshold for ADHD: 0.1627193137276046
Best Model for ADHD: Pipeline(steps=[('scaler', StandardScaler()),
                ('clf',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=1.0, device=None,
                               early_stopping_rounds=None,
                               enable_categorical=False, eval_metric='logloss',
                               feature_types=None, feature_weights=None,
                               gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constra

In [13]:
adhd_scores = []
for t in tqdm(thresholds, desc="ADHD Thresholds"):
    tmp_pred = np.where(adhd_oof > t, 1, 0)
    tmp_score = f1_score(y_adhd, tmp_pred, sample_weight=weights)
    adhd_scores.append(tmp_score)
best_adhd_threshold = thresholds[np.argmax(adhd_scores)]
best_adhd_score = max(adhd_scores)

print( " best_adhd_threshold : " , best_adhd_threshold)

print( "Final Best_ADHD_score", best_adhd_score)


ADHD Thresholds: 100%|██████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 212.98it/s]

 best_adhd_threshold :  0.38383838383838387
Final Best_ADHD_score 0.8853855986365573


In [14]:
best_model_sex.fit(train_combined[features_sex], y_sex, clf__sample_weight=weights)
sex_proba_train = best_model_sex.predict_proba(train_combined[features_sex])[:,1]
sex_proba_test = best_model_sex.predict_proba(test_combined[features_sex])[:,1]

In [15]:
train_combined["sex_proba"] = sex_proba_train
test_combined["sex_proba"] = sex_proba_test
for interaction in interactions:
    train_combined[f"I_{interaction}"] = train_combined["sex_proba"] * train_combined[interaction]
    test_combined[f"I_{interaction}"] = test_combined["sex_proba"] * test_combined[interaction]
best_model_adhd.fit(train_combined[features_adhd], y_adhd, clf__sample_weight=weights)
adhd_proba_test = best_model_adhd.predict_proba(test_combined[features_adhd])[:,1]

In [16]:
submission = pd.read_excel("./SAMPLE_SUBMISSION.xlsx")
submission["ADHD_Outcome"] = np.where(adhd_proba_test > best_adhd_threshold, 1, 0)
submission["Sex_F"] = np.where(sex_proba_test > best_sex_threshold, 1, 0)
submission.to_csv("submission__trial_3.csv", index=False)